In [ ]:
# Librarylist🤖

import glob, warnings, polars as pl, datetime, sqlite3, time, os, zipfile, xml.dom.minidom
from datetime import datetime as dt, time as t, timedelta, date
import pandas as pd, numpy as np, sqlalchemy as sa, xlsxwriter
from sqlalchemy import create_engine
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import win32com.client, xlwings as xw, psutil
# -----------------------------------------------------------------------------------------------#
# Source collection
user_credential = os.path.join(os.environ['USERPROFILE'],r'Concentrix Corporation//CNXVN - WFM Team - Documents//')
# [BKN]SEAT_MAP 📑
PRE_SEAT_Link = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//PRE_SEAT_DATA')
SEAT_MAP_Link = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//SEAT_MAP_DATA')
Seat_Declare = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//Seat_Declare.xlsx')
Map_path = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//Map.xlsb')
SeatMap_path = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//SeatMap.xlsb')
# -----------------------------------------------------------------------------------------------#
# Database Connection
Link_DB = os.path.join(os.environ['USERPROFILE'], r'Desktop//Bcom_DB.db')
conn = create_engine(f"sqlite:///{Link_DB}")
server_name = "PHMANVMDEV01V"
server_ip = "10.5.11.60"
database = "wfm_vn_dev"
user = "usr_wfmvn_dev"
password = "12guWU2OdEj5kEspl9Rlfoglf"
# SQL Server Authentication 🔗
connection_string = f"mssql+pyodbc://{user}:{password}@{server_ip}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
# Windows Authentication 🔗
# connection_string = f"mssql+pyodbc://{server_name}/{database}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
engine = create_engine(connection_string, fast_executemany=True)

In [ ]:
# Get_Roster🗓️

sql_query = """ 

WITH
TL AS (SELECT [Employee_ID],[TED Name] from BCOM.Staff),
OT AS (SELECT [Emp ID],[Date],[OT] from BCOM.RegisteredOT),
ROSTER AS (
SELECT BCOM.ROSTER.[Attribute] as [Date],BCOM.ROSTER.[Emp ID],BCOM.Staff.[TED Name],[LOB],
CASE WHEN OT.[OT]>0 AND BCOM.ROSTER.[Value]='OFF' then BCOM.ROSTER.[week_shift]
	 WHEN BCOM.ROSTER.[Value]='PEGA' THEN '1300-2200'
	 WHEN BCOM.ROSTER.[Value]='Training' and BCOM.ROSTER.[LOB] in ('VICSP','Senior VICSP') THEN '0900-1800'
	 WHEN BCOM.ROSTER.[Value]='HAL' THEN BCOM.ROSTER.[week_shift]
ELSE BCOM.ROSTER.[Value] END AS [Shift],
TL.[TED Name] as [TL_Name],
ISNULL(OT.[OT],0) as [OT],
CASE WHEN BCOM.ROSTER.[Value] IN ('OFF', 'AL', 'CO', 'HO', 'UPL', 'VGH') THEN 'OFF'
	 WHEN BCOM.ROSTER.[Value] IN ('0000-0900', '0100-1000', '0200-1100', '0300-1200', '0400-1300', '0500-1400', '0600-1500', '0700-1600', '0800-1700', '0900-1800', '1000-1900', '1100-2000', '1200-2100', '1300-2200', '1400-2300') THEN 'DS'
	 WHEN BCOM.ROSTER.[Value] IN ('1500-0000', '1600-0100', '1700-0200', '1800-0300', '1900-0400', '2000-0500', '2100-0600', '2200-0700', '2300-0800') THEN 'NS'
	 ELSE Null END AS [Shift_type],
upper(left(DATENAME(weekday, BCOM.ROSTER.[Attribute]),3)) as [Week_day],
BCOM.Staff.[Booking Email]
FROM BCOM.ROSTER
LEFT JOIN BCOM.Staff ON BCOM.Staff.[Employee_ID]=BCOM.ROSTER.[Emp ID]
LEFT JOIN OT ON OT.[Emp ID]=BCOM.ROSTER.[Emp ID] and OT.[Date]=BCOM.ROSTER.[Attribute]
LEFT JOIN TL ON TL.[Employee_ID]=BCOM.ROSTER.[team_leader])
select [Date],[Emp ID],[TED Name],[Shift],[TL_Name],[OT],[Shift_type],[Week_day],[Booking Email],
CASE WHEN [Shift_type]='OFF' then 
DATEADD(second, 0,
DATEADD(SECOND,
DATEDIFF(SECOND,CAST('00:00:00' AS TIME), TIMEFROMPARTS(TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT),TRY_CAST(SUBSTRING([Shift], 3, 2) AS INT), 0, 0, 0)),
CAST([Date] AS DATETIME)))
WHEN [LOB] NOT IN ('EN','VICSP','VICSG','Senior VICSP') and [Shift] ='1200-2100' then
DATEADD(second, 0,
DATEADD(SECOND,
DATEDIFF(SECOND,CAST('00:00:00' AS TIME), TIMEFROMPARTS(TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT),TRY_CAST(SUBSTRING([Shift], 3, 2) AS INT), 0, 0, 0)),
CAST([Date] AS DATETIME)))
else 
DATEADD(second, -[OT]*3600,
DATEADD(SECOND,
DATEDIFF(SECOND,CAST('00:00:00' AS TIME), TIMEFROMPARTS(TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT),TRY_CAST(SUBSTRING([Shift], 3, 2) AS INT), 0, 0, 0)),
CAST([Date] AS DATETIME))) end as [Start time],

CASE WHEN [Shift_type]='OFF' then 
DATEADD(second, 
(case when TRY_CAST(SUBSTRING([Shift], 6, 2) AS INT) > TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT) then (TRY_CAST(SUBSTRING([Shift], 6, 2) AS INT) - TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT))*3600 else (TRY_CAST(SUBSTRING([Shift], 6, 2) AS INT) - TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT)+24)*3600 end),
DATEADD(SECOND,
DATEDIFF(SECOND,CAST('00:00:00' AS TIME), TIMEFROMPARTS(TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT),TRY_CAST(SUBSTRING([Shift], 3, 2) AS INT), 0, 0, 0)),
CAST([Date] AS DATETIME)))

WHEN [LOB] NOT IN ('EN','VICSP','VICSG','Senior VICSP') and [Shift] ='1900-0400' then
DATEADD(second, 
(case when TRY_CAST(SUBSTRING([Shift], 6, 2) AS INT) > TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT) then (TRY_CAST(SUBSTRING([Shift], 6, 2) AS INT) - TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT))*3600 else (TRY_CAST(SUBSTRING([Shift], 6, 2) AS INT) - TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT)+24)*3600 end),
DATEADD(SECOND,
DATEDIFF(SECOND,CAST('00:00:00' AS TIME), TIMEFROMPARTS(TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT),TRY_CAST(SUBSTRING([Shift], 3, 2) AS INT), 0, 0, 0)),
CAST([Date] AS DATETIME)))

ELSE DATEADD(second, 
(case when TRY_CAST(SUBSTRING([Shift], 6, 2) AS INT) > TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT) then (TRY_CAST(SUBSTRING([Shift], 6, 2) AS INT) - TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT)+[OT])*3600 else (TRY_CAST(SUBSTRING([Shift], 6, 2) AS INT) - TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT)+24+[OT])*3600 end),
DATEADD(SECOND,
DATEDIFF(SECOND,CAST('00:00:00' AS TIME), TIMEFROMPARTS(TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT),TRY_CAST(SUBSTRING([Shift], 3, 2) AS INT), 0, 0, 0)),
CAST([Date] AS DATETIME)))
END as [End time],

CASE WHEN [LOB] IN ('ES CSP','IT CSP','DE CSP', 'FR CSP','VICSP','VICSG','Senior VICSP') THEN 0 ELSE 1 END AS [Check],[LOB]

from ROSTER
where [Date]=DATEADD(DAY, +1,CAST(GETDATE() As Date)) and ([Shift_type]<>'OFF' or [Shift_type] is null or [OT]>0) and [Shift] not in ('Training','UPL','AL','CO')
order by [Check],[Start time],[TL_Name] asc

"""    
# Execute the query and load data into a Polars DataFrame, then convert to Pandas
Roster = pl.read_database(query=sql_query, connection=engine)
engine.dispose()
Roster = Roster.to_pandas()
# Final data cleaning and formatting
Roster = Roster[['Date','Emp ID','TED Name','Shift','TL_Name','OT','Shift_type','Week_day','Start time','End time','Booking Email','LOB']]
Roster[['Start time', 'End time']] = Roster[['Start time', 'End time']].apply(pd.to_datetime, format='mixed')
# Export
display(Roster)

In [ ]:
#Get_SeatData🪑

# Import source files
today = pd.to_datetime("today")
dday = today.strftime('%Y-%m-%d')
date_1 = today + pd.Timedelta(days=1)
date_1 = date_1 .strftime('%Y-%m-%d')
# Construct filename for the previous day's seat data
filename="Previous_Seat_Data_" + dday
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet._reader")
# Read seat information from the 'Seat_Declare.xlsx' file
seat_no = pd.read_excel(Seat_Declare,sheet_name="Seat No") # List of all seats
working_hrs=pd.read_excel(Seat_Declare,sheet_name="Working Hours") # Standard operating hours for seat areas
trainee_seat=pd.read_excel(Seat_Declare,sheet_name="Trainee_Seat") # Special rules for trainee seats
# Read the output from the previous day to know when seats were last occupied
seat_no_update = pd.read_excel(PRE_SEAT_Link+"//"+filename+".xlsx",sheet_name="SEAT NO")
# Prepare trainee seat data by creating a full datetime object
trainee_seat['Working hours (Open)']=trainee_seat['Working hours (Open)'].astype("str")
trainee_seat['Datetime']=date_1+' '+trainee_seat['Working hours (Open)']
trainee_seat['Datetime']=pd.to_datetime(trainee_seat['Datetime'])
# Calculate Seat Operating Hours for the target date
seat_no['Date'] = pd.to_datetime(Roster['Date'].min(), format='mixed')
Roster_date = Roster[['Date', 'Week_day']].groupby(['Date', 'Week_day'], as_index=False).count()
working_hrs = pd.merge(working_hrs, Roster_date, on='Week_day', how='left')
working_hrs = working_hrs.dropna(subset=['Date'])
working_hrs['Date'] = pd.to_datetime(working_hrs['Date'])
# Create 'Open Hours' and 'Closed Hours' datetime columns from strings
working_hrs['Open Hours'] = pd.to_datetime(working_hrs['Date'].astype(str) + ' ' + working_hrs['Working hours (Open)'].astype(str), format='%Y-%m-%d %H:%M:%S')
working_hrs['Closed Hours'] = pd.to_datetime(working_hrs['Date'].astype(str) + ' ' + working_hrs['Working hours (Closed)'].astype(str), format='%Y-%m-%d %H:%M:%S')
# Handle cases where closing hour is on the next day
working_hrs['Check'] = pd.to_datetime(working_hrs['Date'].astype(str) + ' 23:59:59', format='%Y-%m-%d %H:%M:%S')
working_hrs['Closed Hours'] = working_hrs.apply(lambda x: x['Closed Hours'] + pd.Timedelta(days=1) if x['Closed Hours'] == x['Check'] else x['Closed Hours'], axis=1)
working_hrs = working_hrs.drop(columns=['Working hours (Open)', 'Working hours (Closed)', 'Floor', 'Building', 'Week_day', 'Check'])
# Merge the calculated working hours with the main seat list
seat_no=pd.merge(seat_no,working_hrs,left_on=['Code','Date'],right_on=['Code','Date'],how='left')
# Merge with previous day's data to get the last occupied time
seat_no=pd.merge(seat_no,seat_no_update[['Seat No','Open Hours']],left_on=['Seat No'],right_on=['Seat No'],how='left')
# A seat's availability for today starts either at its standard open time OR when the previous person's shift ended, whichever is LATER.
seat_no['Open Hours']=seat_no.apply(lambda x: x['Open Hours_y'] if x['Open Hours_y'] > x['Open Hours_x'] else x['Open Hours_x'], axis=1)
seat_no=seat_no.drop(columns=['Open Hours_x','Open Hours_y'])
# Create a check flag to prioritize QTSC9 building
seat_no['Check']=seat_no.apply(lambda x: 1 if x['Building'] =="QTSC9" else 0, axis=1)
# Apply special rules for trainee seats
seat_no=pd.merge(seat_no,trainee_seat,left_on=['Seat No'],right_on=['Seat No'],how='left')
seat_no['Open Hours']=seat_no.apply(lambda x: x['Datetime'] if x['Trainee seat'] =='Y' else x['Open Hours'], axis=1)
# Sort seats to control the allocation order (QTSC9 first, then by earliest availability)
seat_no=seat_no.sort_values(by=['Check','Open Hours','Seat No'],ascending=False)
seat_no=seat_no[['Seat No','Floor','Building','Code','Date','Open Hours','Closed Hours']]
# Display the prepared seat data
display(seat_no)
seat_no.info()

In [ ]:
#Mapping Process🧩

# Initialize DataFrames for the mapping logic
warnings.filterwarnings("ignore", category=FutureWarning)
ghe = seat_no
nhan_vien = Roster
# --- Main Mapping Loop ---
# This process is repeated 4 times to allow multiple agents (with non-overlapping shifts) to be assigned to the same seat.
# For example, a day-shift agent and a night-shift agent can share one seat.

# Loop 1: First pass of seat allocation
# Define a function to find suitable seats for an employee
def tim_ghe_phu_hop(ca_bat_dau, ca_ket_thuc,teamlead,LOB): # "find_suitable_seat(shift_start, shift_end, ...)"
# Special rule: Specific TLs/LOBs must be seated in area 'Q014A'
    if teamlead in ('John Luu','Anita Tran') or LOB in ('ES CSP','IT CSP','FR CSP','DE CSP','VICSG','VICSP','Senior VICSP'):
        return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc) & (ghe['Code']=='Q014A')]['Seat No'].tolist()
# General rule: Find any seat that is available for the entire duration of the shift
    else:
        return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc)]['Seat No'].tolist()
# Apply the function to find a list of all possible seats for each employee
nhan_vien['ghe_phu_hop'] = nhan_vien.apply(lambda row: tim_ghe_phu_hop(row['Start time'], row['End time'],row['TL_Name'],row['LOB']), axis=1)
# Allocate the seats
ghe_da_chon = [] # Keep track of seats that have been taken in this loop
nhan_vien['ghe_duoc_chon'] = None # "selected_seat"
for i, row in nhan_vien.iterrows():
   for ghe_phu_hop in row['ghe_phu_hop']: # "suitable_seat"
           if ghe_phu_hop not in ghe_da_chon:                   
                nhan_vien.loc[i, 'ghe_duoc_chon'] = ghe_phu_hop
                ghe_da_chon.append(ghe_phu_hop)
                break # Assign the first available seat and move to the next employee
# Update the 'Open Hours' of the seats that were just assigned.
# The seat is now available starting from the end of the assigned employee's shift.
ghe=ghe.merge(nhan_vien[['Emp ID','ghe_duoc_chon','End time']],left_on='Seat No', right_on='ghe_duoc_chon', how='left').fillna(0)
ghe['Open Hours']=ghe.apply(lambda x: x['End time'] if x['Emp ID'] !=0 else x['Open Hours'], axis=1)
ghe=ghe[['Seat No','Floor','Building','Code','Date','Open Hours','Closed Hours']]
nhan_vien=nhan_vien[['Date','Emp ID','TED Name','Shift','TL_Name','OT','Shift_type','Week_day','Start time','End time','Booking Email','LOB','ghe_duoc_chon']]
nhan_vien=nhan_vien.rename(columns={'ghe_duoc_chon':'Seat No'})

# Loop 2 to 4: Repeat the process for employees who still don't have a seat
for _ in range(4):  
# (The same logic as Loop 1 is repeated here, but it only tries to assign seats to employees who don't have one yet)
    def tim_ghe_phu_hop(ca_bat_dau, ca_ket_thuc,teamlead,LOB):
        if teamlead in ('John Luu','Layla Phan','Anita Tran') or LOB in ('ES CSP','IT CSP','FR CSP','DE CSP','VICSG','VICSP','Senior VICSP'):
            return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc) & (ghe['Code']=='Q014A')]['Seat No'].tolist()
        else:
            return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc)]['Seat No'].tolist()

    nhan_vien['ghe_phu_hop'] = nhan_vien.apply(lambda row: tim_ghe_phu_hop(row['Start time'], row['End time'],row['TL_Name'],row['LOB']), axis=1)

    ghe_da_chon = []
    nhan_vien['ghe_duoc_chon'] = None
    for i, row in nhan_vien.iterrows():
       for ghe_phu_hop in row['ghe_phu_hop']:
           if ghe_phu_hop not in ghe_da_chon:
               if nhan_vien.loc[i,'Seat No'] == None:
                   nhan_vien.loc[i, 'ghe_duoc_chon'] = ghe_phu_hop
                   ghe_da_chon.append(ghe_phu_hop)           
                   break
# Update seat availability again
    ghe=ghe.merge(nhan_vien[['Emp ID','ghe_duoc_chon','End time']],left_on='Seat No', right_on='ghe_duoc_chon', how='left')
    ghe['Emp ID'] = ghe['Emp ID'].fillna(0)
    ghe['Open Hours']=ghe.apply(lambda x: x['End time'] if x['Emp ID'] !=0 else x['Open Hours'], axis=1)
    ghe=ghe[['Seat No','Floor','Building','Code','Date','Open Hours','Closed Hours']]
# Update the main employee list with newly assigned seats
    nhan_vien['Seat No']=nhan_vien.apply(lambda x: x['ghe_duoc_chon'] if x['Seat No'] ==None else x['Seat No'], axis=1)
    nhan_vien=nhan_vien[['Date','Emp ID','TED Name','Shift','TL_Name','OT','Shift_type','Week_day','Start time','End time','Booking Email','LOB','Seat No']]

# --- Finalization and Export ---
nhan_vien['Emp ID']=nhan_vien['Emp ID'].astype("string")
# Create a list of employees who couldn't be assigned a seat
no_seat_list=nhan_vien.loc[(nhan_vien['Seat No'].isnull()==True)]
# Add floor and building info to the final employee list
nhan_vien=nhan_vien.merge(ghe[['Seat No','Floor','Building']],left_on='Seat No', right_on='Seat No', how='left')
# Create a list of available (unoccupied) seats
occupied_seat = nhan_vien[['Seat No']].drop_duplicates()
occupied_seat.columns = ['Seat']
seat_available = pd.merge(ghe, occupied_seat, left_on='Seat No', right_on='Seat', how='left')
seat_available = seat_available.loc[seat_available['Seat'].isnull()]
# Create a pivot table for an overview report
pivot_seat=nhan_vien[['TED Name','Shift','TL_Name','Seat No','Floor','Building']]

#Convert to Pandas
nhan_vien = pl.from_pandas(nhan_vien)
ghe = pl.from_pandas(ghe)
no_seat_list = pl.from_pandas(no_seat_list)
seat_available = pl.from_pandas(seat_available)
pivot_seat = pl.from_pandas(pivot_seat)
pivot_seat = pivot_seat.pivot(index=['TL_Name', 'Shift'],on=['Building', 'Floor'],values='TED Name',aggregate_function='len')
pivot_seat = pivot_seat.sort(['TL_Name', 'Shift'])

# Export to CSV
def write_to_excel(filename, data_dict):
  with xlsxwriter.Workbook(filename) as workbook:
    for worksheet_name, data in data_dict.items():
      worksheet = workbook.add_worksheet(worksheet_name)
      data.write_excel(workbook=workbook, worksheet=worksheet_name, position="A1")
# Write the main SEAT_MAP output file
seat_map_filename="SEAT_MAP_" + date_1
seat_data = {
    'OVERVIEW': pivot_seat,'SEAT MAP': nhan_vien,'AGENTS NO SEAT': no_seat_list,'SEAT AVAILABLE': seat_available,
}
write_to_excel(SEAT_MAP_Link+"//"+seat_map_filename+".xlsx", seat_data)
# Write the 'Previous_Seat_Data' file for the next day's run
os.chdir(PRE_SEAT_Link)
pre_seat_data_filename = "Previous_Seat_Data_" + date_1
pre_seat_data = ghe.write_excel(pre_seat_data_filename, worksheet = "SEAT NO")

# Display results
display(no_seat_list)
display(nhan_vien)
display(seat_available)

In [ ]:
#Map_path🔃

# Use win32com to interact with the Excel application COM object
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False # Run Excel in the background without showing the UI
# Use xlwings to open the specific workbook
wb = xw.Book(Map_path)
# Execute the VBA macro named "Refresh" within the workbook
wb.macro("Refresh")()
# Save and close the workbook
wb.save()
# This part forcefully terminates the EXCEL.EXE process. 
for proc in psutil.process_iter(['pid', 'name']):
    if proc.info['name'] == "EXCEL.EXE":
        proc.kill()

In [ ]:
#SeatMap_path🔃

# Use win32com to interact with the Excel application COM object
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False # Run Excel in the background without showing the UI
# Use xlwings to open the specific workbook
wb = xw.Book(SeatMap_path)
# Execute the VBA macro named "Refresh" within the workbook
wb.macro("Refresh")()
# Save and close the workbook
wb.save()
# This part forcefully terminates the EXCEL.EXE process. 
for proc in psutil.process_iter(['pid', 'name']):
    if proc.info['name'] == "EXCEL.EXE":
        proc.kill()